In [1]:
from HGCal_ImageAlgo3D import *
from utility_benchmark import *
from timeit import default_timer as timer

%matplotlib inline
plt.style.use('classic')
plt.rc("figure",facecolor="w",figsize=(6,4))
plt.rc("font",size=10)
plt.rc("savefig",dpi=300)

DatasetDir  = '../data/'
DatasetFile = 'CMSSW93X_pt35_pu000'

Welcome to JupyROOT 6.10/00


In [2]:
df = pd.read_pickle(DatasetDir+"input/"+DatasetFile+"_rechit.pkl")
dfgen = pd.read_pickle(DatasetDir+"input/"+DatasetFile+"_gen.pkl")

# 1. Define Fitness

In [3]:
def TranslateDNA(DNA):
    para_upper = np.array([2,8,8,5])
    para_lower = np.array([0,2,2,2])
    para = DNA*(para_upper-para_lower)+para_lower
    return para

def DefineFitness(DNA):
    para = TranslateDNA(DNA)
    ia=ImagingAlgo3D(MAXDISTANCE        = 200,  #cm
                     LAYER_DISTANCE     = para[0], #1.2,  #cm
                     KERNAL_R           = para[1], #4.0,  #cm
                     KERNAL_R_NORM      = para[2], #2.0,  #cm
                     KERNAL_R_POWER     = 2.0,  #cm
                     KERNAL_LAYER       = 2.0,  #number of layer
                     DECISION_RHO_KAPPA = 10.0, #fractio of max rho
                     DECISION_NHD       = para[3], #4.0,  #cm
                     CONTINUITY_NHD     = 6.0   #cm 
                    )

    dfresultclus = ia.RunImagingAlgo(df,Nevent=10,verb=False)
    bm = Benchmark(DatasetFile, dfresultclus, N=10)
    sigma,mean = bm.getEffSigma_EnergyEfficiency(deltarho = 10)
    
    fitness = 1/sigma
    return fitness

# 2. Define GA and Optimize

In [4]:
from GeneticOptimizer import *
from IPython.display import clear_output

ga = GA(pop_size    = 100,
        DNA_size    = 4, 
        DNA_fitness = DefineFitness,
        cross_rate  = 0.8,
        mutate_rate = 0.01,
        noise_rate  = 0.4,
        noise_amp   = 0.02
       )

In [ ]:
fitness_list = []
best_para_list = []
for generation in range(2):
    print(generation)
    clear_output(wait=True)
    
    ga.evolve()
    fitness  = ga.get_pop_fitness()
    best_DNA = ga.pop[np.argmax(fitness)]
    fitness_list.append(fitness)
    best_para_list.append(TranslateDNA(best_DNA))
np.save('../data/fitness_pu000',np.array(fitness_list))
np.save('../data/bestpara_pu000',np.array(best_para_list))

0


In [ ]:
f = np.load('../data/fitness_pu000_100.npy')
p = np.load('../data/bestpara_pu000_100.npy')

In [1]:
sigma = 1/(np.max(f,axis=1))
#sigma = sigma.reshape(-1,2)
#sigma = np.mean(sigma, axis=1)
evolve = np.arange(0,20,1)
plt.plot(evolve,sigma,'c',lw=2,label='optimal in a generation')
#plt.plot(evolve,sigma_pred,'k--',lw=1)
#plt.axhline(0.1165)
plt.legend()
plt.grid()
plt.xlabel('Generation of Evolution',fontsize=14)
plt.ylabel(r'$\sigma^{eff}_\epsilon$',fontsize=14)
#plt.savefig('../plot/ga.png',dpi=300)